In [11]:
import time
import urx
import logging
import numpy as np
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
actions = np.array(['Start','Go','Stop'])
model = Sequential()
model.add(LSTM(64,return_sequences=True, activation='relu', input_shape=(50,63)))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(64,return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.load_weights('gesture.h5')

In [14]:
def draw_hlandmarks(frame,results):
    for num,hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(frame,hand,mp_hands.HAND_CONNECTIONS) #rendering
def extract_keypoints(results):
    if results.multi_hand_landmarks:
        return np.array([[lmark.x,lmark.y,lmark.z] for lmark in results.multi_hand_landmarks[0].landmark]).flatten() if results.multi_hand_landmarks[0].landmark else np.zeros((21*3))
    else:
        return np.zeros((21*3))

In [ ]:
def moverobot(actionresult):
    if actionresult == "Stop":
        rob.stopj(a)
    
    elif actionresult == "Start":
        l = 0.05
        v = 0.07
        a = 0.1
        pose = rob.getl()
        pose[0] += l
        rob.movej(pose, acc=a, vel=v, wait=True)
    
    elif actionresult == "Go":
        l = 0.05
        v = 0.07
        a = 0.1
        r = 0.05
        pose = rob.getl()
        pose[1] += l
        rob.movej(pose, acc=a, vel=v, wait=True)
        

In [17]:
if __name__ == "__main__":
    rob = urx.Robot("192.168.1.6")
    try:
        l = 0.1
        v = 0.07
        a = 0.1
        r = 0.05
        pose = rob.getl()
        pose[2] += l
        rob.movep(pose, acc=a, vel=v, radius=r, wait=False)
        while True:
            p = rob.getl(wait=True)
            if p[2] > pose[2] - 0.05:
                break

        pose[1] += l 
        rob.movep(pose, acc=a, vel=v, radius=r, wait=False)
        while True:
            p = rob.getl(wait=True)
            if p[1] > pose[1] - 0.05:
                break

        pose[2] -= l
        rob.movep(pose, acc=a, vel=v, radius=r, wait=False)
        while True:
            p = rob.getl(wait=True)
            if p[2] < pose[2] + 0.05:
                break

        pose[1] -= l
        rob.movep(pose, acc=a, vel=v, radius=0, wait=True)

    finally:
        rob.close()

TimeoutError: timed out

In [24]:
#Testing in Real Time
rob = urx.Robot("192.168.1.6")
sequence = []
sentence = []
threshold = 0.9
actionresult = actions[0]
vidcap = cv2.VideoCapture(0)
try:
    with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=1) as hands:
        while vidcap.isOpened():
            ret,frame = vidcap.read()
            image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)#OpenCV works in BlueGreenRed format but Mediapipe works in RedGreenBlue format
            image.flags.writeable = False #setting flags to prevent copying the image
            results = hands.process(image) #detection
            image.flags.writeable = True #setting it back
            image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #changing back
            #print(results) #output all the landmarks 
            if results.multi_hand_landmarks:
                draw_hlandmarks(frame,results) #rendering

            #predictions
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-50:]
            
            if len(sequence) == 50:
                res = model.predict(np.expand_dims(sequence,axis=0))[0]
                if res[np.argmax(res)] > threshold:
                    if actionresult != actions[np.argmax(res)]:
                        actionresult = actions[np.argmax(res)]
                        print(actionresult)
                        moverobot(newres)

            cv2.imshow('Hand Tracking',frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        vidcap.release()
        cv2.destroyAllWindows()
finally:
    rob.close()

1/1 [==============================] - 0s 30ms/step
Stop
1/1 [==============================] - 0s 27ms/step
Start
1/1 [==============================] - 0s 25ms/step


In [13]:
vidcap.release()
cv2.destroyAllWindows()